In [1]:
import pandas as pd
from projects.ccs.rooftop_solar_project import RooftopSolarProject



In [2]:
all_df=pd.read_csv('/Volumes/Samsung_T5/data/ccs/6kw_solar_npc_states_3tilts_9azimuths.csv',index_col=[0])

In [3]:
time_varying_carbon_intensity_df=pd.read_csv('/Volumes/Samsung_T5/data/ccs/us_fossil_fuel_carbon_intensity_projections.csv')

In [16]:
shared_params={}
shared_params['inflation_rate']=0.025
shared_params['discount_rate']=0.12
shared_params['tco2_per_kwh']=list(time_varying_carbon_intensity_df['carbon intensity'])
shared_params['project_length_yrs']=25
shared_params['bond_interest_rate']=0.0428

In [17]:
s=RooftopSolarProject(all_df.loc[1,['kwh_per_yr', 'lat', 'lon', 'state', 'tilt', 'azimuth',
       'kw', 'region',
       'installation_cost_usd', 'usd_per_kwh']].to_dict() | shared_params)

In [18]:
install_df=pd.read_csv()

10014.084443986128

In [27]:
for i,row in all_df.loc[all_df.installation_cost_scenario=='low_install_usd'].iterrows():
       s=RooftopSolarProject(row[['kwh_per_yr', 'lat', 'lon', 'state', 'tilt', 'azimuth',
       'kw', 'region',
       'installation_cost_usd', 'usd_per_kwh']].to_dict() | shared_params)
       all_df.at[i,'pv_solar_usd_per_tco2']=s.pv_solar_usd_per_tco2
       all_df.at[i,'pv_expense_usd_per_tco2']=s.pv_expense_usd_per_tco2
       all_df.at[i,'npv']=s.npv
       all_df.at[i,'total_co2']=s.total_tco2

       


In [28]:
all_df.columns

Index(['kwh_per_yr', 'lat', 'lon', 'state', 'tilt', 'azimuth', 'kw', 'region',
       'tco2_per_kwh', 'installation_cost_scenario', 'installation_cost_usd',
       'usd_per_kwh', 'pv_solar_usd_per_tco2', 'pv_expense_usd_per_tco2',
       'npv', 'total_co2'],
      dtype='object')

In [29]:
ccs_df=pd.read_csv('/Volumes/Samsung_T5/data/ccs/ues_simulations.csv',index_col=[0])

In [30]:
ccs_df

,industry,well_type,brent_since_yr,simulation_date,mean_oil_price,oil_breakeven_price,scenario,pv_gs_subsidy_unit_revenue_usd_per_tco2,pv_eor_subsidy_unit_revenue_usd_per_tco2,total_eor_usd_per_tco2,total_gs_usd_per_tco2
0,NG Processing,existing,1987,2024-06-27,67.231368,55.888028,realistic simulations for testing ccs profitab...,41.257574,29.122993,1.118162,-14.165739
1,Ethanol,existing,1987,2024-06-27,54.454741,38.142184,realistic simulations for testing ccs profitab...,41.257574,29.122993,-6.209385,-22.060831
2,Ammonia,existing,1987,2024-06-27,83.549366,48.912952,realistic simulations for testing ccs profitab...,41.257574,29.122993,7.257347,-32.493204
3,Hydrogen,existing,1987,2024-06-27,80.176747,35.379498,realistic simulations for testing ccs profitab...,41.257574,29.122993,-9.908914,-55.658592
4,Cement,existing,1987,2024-06-27,66.043159,42.529600,realistic simulations for testing ccs profitab...,41.257574,29.122993,-50.853507,-62.737818
...,...,...,...,...,...,...,...,...,...,...,...
63995,Hydrogen,new,2014,2024-06-27,81.814295,67.483511,realistic simulations for testing ccs profitab...,41.257574,29.122993,-70.832685,-66.319698
63996,Cement,new,2014,2024-06-27,84.717325,81.469384,realistic simulations for testing ccs profitab...,41.257574,29.122993,-89.576569,-82.082938
63997,Refinery,new,2014,2024-06-27,77.534297,50.015625,realistic simulations for testing ccs profitab...,41.257574,29.122993,-91.995749,-122.637311
63998,Iron/Steel,new,2014,2024-06-27,79.824407,53.988259,realistic simulations for testing ccs profitab...,41.257574,29.122993,-52.993022,-74.582543


In [31]:
all_df

,kwh_per_yr,lat,lon,state,tilt,azimuth,kw,region,tco2_per_kwh,installation_cost_scenario,installation_cost_usd,usd_per_kwh,pv_solar_usd_per_tco2,pv_expense_usd_per_tco2,npv,total_co2
0,8204.322296,32.318231,-86.902298,AL,18,105.00,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,138.647016,103.187592,3441.243868,97.047371
1,8511.235356,32.318231,-86.902298,AL,18,123.75,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,138.647016,99.466672,3944.590255,100.677787
2,8753.698517,32.318231,-86.902298,AL,18,142.50,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,138.647016,96.711608,4342.236923,103.545838
3,8913.110541,32.318231,-86.902298,AL,18,161.25,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,138.647016,94.981910,4603.677301,105.431492
4,8968.018854,32.318231,-86.902298,AL,18,180.00,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,138.647016,94.400366,4693.728539,106.080992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2344,8581.146036,29.760400,-95.369800,TXE,36,217.50,6,gulf_coast,0.000344,avg_install_usd,16560.0,0.1490,130.172686,111.139361,1931.972929,101.504747
2345,8242.116886,29.760400,-95.369800,TXE,36,236.25,6,gulf_coast,0.000344,avg_install_usd,16560.0,0.1490,130.172686,115.710939,1409.939916,97.494436
2346,8242.116886,29.760400,-95.369800,TXE,36,236.25,6,gulf_coast,0.000344,avg_install_usd,16560.0,0.1490,130.172686,115.710939,1409.939916,97.494436
2347,7808.018567,29.760400,-95.369800,TXE,36,255.00,6,gulf_coast,0.000344,avg_install_usd,16560.0,0.1490,130.172686,122.144059,741.520538,92.359569


In [32]:
import numpy_financial as npf


In [33]:
npf.pmt(0.05,20,17395,0)

-1395.819804182075

In [35]:
all_df[['kwh_per_yr','region',
       'tco2_per_kwh', 'installation_cost_usd',
       'usd_per_kwh', 'pv_solar_usd_per_tco2', 'pv_expense_usd_per_tco2',
       'npv', 'total_co2']].loc[all_df.installation_cost_scenario=='low_install_usd'].groupby('region').mean()

,kwh_per_yr,tco2_per_kwh,installation_cost_usd,usd_per_kwh,pv_solar_usd_per_tco2,pv_expense_usd_per_tco2,npv,total_co2
region,,,,,,,,
california,9436.696643,0.000178,16666.000000,0.383567,335.100023,101.998632,26002.450833,111.624893
gulf_coast,8608.179398,0.000359,15738.000000,0.142657,124.631299,105.663122,1973.774917,101.824520
midwest,7772.769113,0.000462,16838.000000,0.164822,143.995647,125.726260,1693.068297,91.942610
north_central,9018.704808,0.000519,15404.285714,0.119857,104.712257,99.041766,693.769883,106.680547
south_central,9678.883446,0.000322,15720.000000,0.135200,118.116424,93.801131,2870.250792,114.489674
